# Get Programming with F# by [Isaac Abraham](https://github.com/isaacabraham)

## “CAPSTONE 2”

I am seeing Capstone 2 as an opportunity to do two things:

1. download, run and analyze [the source code](https://github.com/isaacabraham/get-programming-fsharp/tree/master/src/code-listings/lesson-14) designed for lesson 13 in the book
2. use `#load` in an F# script of this notebook


## downloading from GitHub

One way to download our code from GitHub is via a [DownGit link](https://downgit.github.io/#/home?url=https://github.com/isaacabraham/get-programming-fsharp/tree/master/src/code-listings/lesson-14). This will allow us to save the GitHub folder as a ZIP archive on our local machine.

Success with DownGit means we should see an `*.fsx` file among several `*.fs` files after extracting the ZIP archive by hand:


In [ ]:
Get-ChildItem -Path ./14-capstone-02 -Name

App.config
Auditing.fs
Capstone2.fsproj
Capstone2.fsx
Capstone2.sln
Domain.fs
Operations.fs
Program.fs


Note that the `*.fsproj` and `*.sln` files shown above might not be in .NET Core formats.


## `Capstone2.fsx` in this notebook

The contents of `Capstone2.fsx` can be copied an pasted below with slight modifications of the `#load` paths:


In [ ]:
#!fsharp

#load "14-capstone-02/Domain.fs"
#load "14-capstone-02/Operations.fs"
#load "14-capstone-02/Auditing.fs"

open Capstone2.Operations
open Capstone2.Domain
open Capstone2.Auditing
open System

// Create console-auditing withdraw and deposit functions.
// Notice that the signatures of the new "shadowed" functions
// have the same signature as the original ones. This is equivalent
// to a Decorator in OO terms.
let withdraw = auditAs "withdraw" console withdraw
let deposit = auditAs "deposit" console deposit

let customer = { Name = "Isaac" }
let account = { AccountId = Guid.Empty; Owner = customer; Balance = 90M }

// Test out withdraw
let newAccount = account |> withdraw 10M
newAccount.Balance = 80M // should be true!

// Test out console auditer
console account "Testing console audit"
// should print "Account 00000000-0000-0000-0000-000000000000: Testing console audit"

account
|> withdraw 50M
|> deposit 50M 
|> deposit 100M 
|> withdraw 50M 
|> withdraw 350M

Account 00000000-0000-0000-0000-000000000000: 10/21/2021 3:13:50 AM: Performing a withdraw operation for £10...
Account 00000000-0000-0000-0000-000000000000: 10/21/2021 3:13:50 AM: Transaction accepted! Balance is now £80.
Account 00000000-0000-0000-0000-000000000000: Testing console audit
Account 00000000-0000-0000-0000-000000000000: 10/21/2021 3:13:50 AM: Performing a withdraw operation for £50...
Account 00000000-0000-0000-0000-000000000000: 10/21/2021 3:13:50 AM: Transaction accepted! Balance is now £40.
Account 00000000-0000-0000-0000-000000000000: 10/21/2021 3:13:50 AM: Performing a deposit operation for £50...
Account 00000000-0000-0000-0000-000000000000: 10/21/2021 3:13:50 AM: Transaction accepted! Balance is now £90.
Account 00000000-0000-0000-0000-000000000000: 10/21/2021 3:13:50 AM: Performing a deposit operation for £100...
Account 00000000-0000-0000-0000-000000000000: 10/21/2021 3:13:50 AM: Transaction accepted! Balance is now £190.
Account 00000000-0000-0000-0000-00000000

AccountId,Owner,Balance
00000000-0000-0000-0000-000000000000,"{ { Name = ""Isaac"" }: Name: Isaac }",140


## two different functions with the same argument signature can be passed like an interface

Let us take a look at `Operations.fs`:


In [ ]:
Get-Content -Path ./14-capstone-02/Operations.fs

module Capstone2.Operations

open System
open Capstone2.Domain

/// Withdraws an amount of an account (if there are sufficient funds)
let withdraw amount account =
    if amount > account.Balance then account
    else { account with Balance = account.Balance - amount }

/// Deposits an amount into an account
let deposit amount account =
    { account with Balance = account.Balance + amount }

/// Runs some account operation such as withdraw or deposit with auditing.
let auditAs operationName audit operation amount account =
    audit account (sprintf "%O: Performing a %s operation for £%M..." DateTime.UtcNow operationName amount)
    let updatedAccount = operation amount account
    
    let accountIsUnchanged = (updatedAccount = account)

    if accountIsUnchanged then audit account (sprintf "%O: Transaction rejected!" DateTime.UtcNow) 
    else audit account (sprintf "%O: Transaction accepted! Balance is now £%M." DateTime.UtcNow updatedAccount.Balance)

    updatedAccount


The `auditAs` function takes an `operation` argument which is a function that is called like this:

```fsharp
let updatedAccount = operation amount account
```

The subtle beauty here is that `withdraw` and `deposit` functions share the signature of `operation` and can be passed into `auditAs` interchangeably! This pattern holds for `audit account` as well, making `auditAs` a [_higher order function_](https://en.wikipedia.org/wiki/Higher-order_function).


### mutating with `with`

Notice that the `withdraw` and `deposit` functions return a _new copy_ of `account` using `with` in an _object expressions_ [📖 [docs](https://docs.microsoft.com/en-us/dotnet/fsharp/language-reference/object-expressions)] which are not mentioned until lesson 25:

In [ ]:
#!fsharp

let account1: Account = { AccountId = Guid.NewGuid(); Owner = { Name = "Ms. One" }; Balance = 577.0m }
let account2: Account = { account1 with Balance = 39.99m }

account1

AccountId,Owner,Balance
5aa1e111-426e-4061-b4a5-63a74cc9d9a0,"{ { Name = ""Ms. One"" }: Name: Ms. One }",577.0


In [ ]:
#!fsharp

account2

AccountId,Owner,Balance
5aa1e111-426e-4061-b4a5-63a74cc9d9a0,"{ { Name = ""Ms. One"" }: Name: Ms. One }",39.99


So we see that `withdraw` and `deposit` are returning copies of `account` to overwrite the `mutable` `account` in `Program.fs`.

@[BryanWilhite](https://twitter.com/BryanWilhite)


In [ ]:
#!about

.NET Interactive© 2020 Microsoft CorporationVersion: 1.0.250604+e99bd250539a64e828b44cb9ad9353eeda9b5bc1Build date: 2021-10-07T01:11:06.0000000Zhttps://github.com/dotnet/interactive
